# Azure ML SDK V2

Nå som Azure ML SDK V2 skal ut av preview, kan det være greit å få testet ut hvordan man oppretter gjenbrukbare komponenter. Som nevnt tidligere, merk at det er et par andre måter dette kan gjøres på med V2, som ikke er releaset i skrivende stund. Så dagens approach blir en script-yaml combo, der scriptet inneholder logikken for hva komponenten din skal gjøre, mens YAML filen definerer hva komponenten er(altså, hva den tar inn, gir ut, hva den trenger for å kjøre etc...). Om du sitter fast, gjerne spør en av oss som holder fagdagen, snakk med sidemannen, eller referer til denne "tutorialen" fra Microsoft om du vil ha det litt lettere https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-component-pipelines-ui . For ordens skyld har vi satt det opp slik at det genereres en filstruktur, og du kan jobbe i notebooken hele tiden. Målet med denne notebooken er å få inn arbeidet du gjorde i del 1 av denne fagdagen inn i Azure Machine Learning Studio som en pipeline

Her har dere et eksempel å gå etter. Denne prep.py filen henter inn argumenter via parser filen. Her er det viktig at det stemmer overens med det dere definerer i yaml filen, da det er det som forteller azure hva som kommer inn og ut av komponenten. Her er prep steget der vi henter inn mnist datasettet, gjør det om fra dataframe til numpy(grunnet mltable kun har to_pandas_dataframe()), og prosesserer det. Så lagrer vi de som npy filer(binaries) på lokasjonen som er definert i output_test(Denne vil bli definert automatisk når du kobler opp pipelinen)

In [19]:
import os

prep_src_dir = "./components/prep"
os.makedirs(prep_src_dir, exist_ok=True)

In [96]:
%%writefile {prep_src_dir}/prep.py

import argparse
import mltable
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os
parser = argparse.ArgumentParser("prep")
parser.add_argument("--dataset", type=str, help="The mltable training data")
parser.add_argument('--output_train', type=str, help='Path of train set')
parser.add_argument('--output_test', type=str, help='Path of test set')

args = parser.parse_args()


tbl_train = mltable.load(args.dataset)
df = tbl_train.to_pandas_dataframe()

X_train_flattened = df.drop("label", axis=1).to_numpy().astype('uint8')
X_train = X_train_flattened.reshape(X_train_flattened.shape[0], 28, 28)
y_train = df["label"].to_numpy().astype('uint8')

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)


# Filter training data 
train_filter = np.where(y_train == 2)
test_filter = np.where(y_test == 2)

X_train, Y_train = X_train[train_filter], y_train[train_filter]
X_test, Y_test = X_test[test_filter], y_test[test_filter]

X_train = X_train.reshape((X_train.shape[0], 28*28)).astype('float32')
X_test = X_test.reshape((X_test.shape[0], 28*28)).astype('float32')
X_train = X_train / 255
X_test = X_test / 255

print(X_train)
print(X_test)

np.save(args.output_train + "/train.npy",  X_train)
np.save(args.output_test + "/test.npy", X_test)



Overwriting ./components/prep/prep.py


For å forsikre oss om at komponentene kjører, så trenger vi en conda.yml fil der du definerer dependencies. Her kan du lage en for hver komponent, eller en stor som skal håndtere alle komponentene. For mindre prosjekter skal dette gå fint, men med mange avhengigheter, så risikerer man fort at det tar lang tid å bygge images, også om du kun har lagt til en dependency for en spesifikk komponent, da det vil trigge nytt bygg for alle.

In [ ]:
%%writefile components/conda.yml
name: demo_env
channels:
  - conda-forge
dependencies:
  - python=3.7.6
  - pip
  - pip:
    - pandas
    - pyjokes
    - mltable
    - argparse
    - azureml-dataprep[pandas]
    - tensorflow
    - scikit-learn
    - numpy
    - azureml-mlflow

Nå som prep scriptet ditt er klart, trenger du å definere en prep.yaml fil som er ment til å fortelle Azure hvordan komponenten din ser ut. Her er YAML definisjonen for prep komponenten

In [69]:
%%writefile ./components/prep.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: prep_INSERT_NAME_HERE
# version: 1
display_name: Preprocessing step for mnist autoencoder
type: command
inputs:
  dataset:
    type: mltable
outputs:
  output_train:
    type: uri_folder
  output_test:
    type: uri_folder
environment:
  conda_file: ./conda.yml
  image: mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220314.v1
code: ./prep
command: >-
  python prep.py 
  --dataset ${{inputs.dataset}} 
  --output_train ${{outputs.output_train}}
  --output_test ${{outputs.output_test}}

Overwriting ./components/prep.yml


## Oppgave 1
Her ønsker vi at dere lager en komponent med **i hvert fall** en input som da vil være treningsdata fra forrige steg. Du kan strukturere resten selv ut ifra hva du vil kunne gi inn til komponenten. Målet med komponenten er å trene en modell, og bruke denne i neste komponent.

In [70]:
import os

train_src_dir = "./components/train"
os.makedirs(train_src_dir, exist_ok=True)

Lag et treningsskript på lignende måte som prep komponenten. Når du har "fungerende" logikk, legg til **%%writefile {train_src_dir}/train.py
** på toppen av skriptet slik at det skrives til treningsmappen(Om du ikke allerede har definert det i .py filer istedenfor notebook). Her står du fritt til hvordan du vil lagre modellen, men vi anbefaler å bruke keras sin save_model (siden dette er en custom modell, så vil du helst bare lagre filen uten extension, da dette vil lage en tensorflow fil som er lettere å laste inn enn f.eks .h5 med mindre du har skrevet kode for det), du kan også bruke [mlflow](https://mlflow.org/) om du vil, men dette vil komme i en senere oppgave.

Overwriting ./components/train/train.py


Definer nå en YAML fil på lignende måte som preprosesseringen

In [84]:
%%writefile components/train.yml


Overwriting components/train.yml


## Oppgave 3
Nå som vi har en treningskomponent, har vi lyst til å score modellen. Avhengig av hvordan du lagret modellen. vil du bruke samme rammeverk til å laste den inn.

In [85]:
import os

score_src_dir = "./components/score"
os.makedirs(score_src_dir, exist_ok=True)

In [86]:
%%writefile {score_src_dir}/score.py


Overwriting ./components/score/score.py


In [87]:
%%writefile ./components/score.yml


Overwriting ./components/score.yml


## Oppgave 4
Nå når du har dine tre komponenter er det klart for å faktisk deploye de. Først må du åpne en terminal og logge deg inn i azure. Prøv først az login --identity om du har en identitet storet, eller om du gjør dette via azure, ellers vil az login ta deg gjennom autentiseringsløypen.

Når dette er gjort kan du endelig lage komponenter. Kjør kommandoen **az ml component create --file "Path to yaml file" ** for hver fil, og trykk deg videre til Designer i ML studio. Her velger du custom under new pipeline, og slå deg løs!


## Oppgave 5
Nå har du en fungerende pipeline, men biten med modellen kan ha vært litt kronglete. Nå skal vi ta i bruk [mlflow](https://mlflow.org/) biblioteket for lagring, lasting og logging av modellen.

In [95]:
%%writefile {train_src_dir}/train.py



Overwriting ./components/train/train.py


In [94]:
%%writefile {score_src_dir}/score.py


Overwriting ./components/score/score.py


## Veien videre
Å definere gjenbrukbare komponenter på denne måten har sine gode og dårlige sider. Noen ønsker gjerne å slippe filer og terminalkommandoer, mens andre foretrekker oppdeligen, da det naturlig kan falle inn i f.eks en CI/CD pipeline. Men for ordens skyld tenkte vi bare å kjapt trekke frem hovedpunktene av en annen variasjon, nemlig å definere hele pipelinen, istedenfor kun komponentene.

```
import os
from pathlib import Path
from mldesigner import command_component, Input, Output


@command_component(
    name="component_name",
    version="1",
    display_name="Display name",
    description="Insert description here",
    environment=dict(
        conda_file=Path(__file__).parent / "conda.yaml",
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    ),
)
def component(
    input_1: Input(type="<Insert Type>"),
    output_1: Output(type="<Insert Type>"),
    output_2: Output(type="<Insert Type>"),
):
    do something
```

Her tar vi i bruk @command_component for å definere denne som en komponent. Merk at vi fremdeles trenger en conda.yaml fil for å definere hva denne komponenten trenger for å kunne kjøre, men du slipper nå å definere selve komponenten.
Kunne alternativt blitt gjort uten decoratoren og med et eget python scrpit du refererer til, ved å lage en vanlig command, og referere til scriptet ditt. Merk at her slipper du også YAML definisjon, da du vil bruke én enkelt conda.yaml for selve environmentet du kjører i, istedenfor å legge dette ved på hver komponent

```
from azure.ai.ml import command
from azure.ai.ml import Input, Output

train_model_component = command(
    name="train",
    display_name="Train",
    inputs={
        "training_data": Input(type = "uri_folder"),
        "max_epocs": Input(type = "integer", min = 0, max= 100),
        "learning_rate": Input(type = "number", default= 0.01) 
    },
    outputs={
    "model_output": Output(type = "uri_folder")
    },
    # The source folder of the component
    code=train_src_dir,
    command="""python train.py \
  --training_data ${{inputs.training_data}} --max_epocs ${{inputs.max_epocs}}   --learning_rate ${{inputs.learning_rate}}  \
  --model_output ${{outputs.model_output}}
            """,
    environment="azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1",
)
```

```
# define a pipeline 
@pipeline(
    default_compute=cpu_compute_target,
)
def pipeline(pipeline_input_data):
    prepare_data_node = prepare_data_component(input_data=pipeline_input_data)

    node1 = component(
        input_data=prepare_data_node.outputs.training_data
    )
    node1.compute = gpu_compute_target

    etc

# create a pipeline
pipeline_job = pipeline(pipeline_input_data=<dataset>)
```

Bruker også her @pipeline for å definere at dette er en pipeline. Om du er mer interessert i å videre utforske dette, sjekk her: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-component-pipeline-python (merk at den ble skrevet samme dag som denne notebooken, så du kan risikere at ting som mldesigner ikke funker...)